## Job 1

In [1]:
%%writefile mapper1.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
#         print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper1.py


In [2]:
%%writefile reducer1.py

import sys

current_key = None
word_sum = 0

Overwriting reducer1.py


In [3]:
%%writefile -a reducer1.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer1.py


## Job 2

In [4]:
%%writefile mapper2.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        word, count = unicode(line.strip()).split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue

    print "%s\t%d" % (word.lower(), count)

Overwriting mapper2.py


In [5]:
%%writefile reducer2.py

import sys

myList = []

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    
    myList.append([count, word])
    
myList.sort(reverse=True)
for l in myList:
    [count, word] = l
    print "%s\t%d" % (word.lower(), count)

Overwriting reducer2.py


## Execute

In [6]:
%%bash

# Job 1
IN_DIR_1="/data/wiki/en_articles_part"
OUT_DIR_1="first_job_result"
NUM_REDUCERS_1=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR_1} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="First job" \
    -D mapreduce.job.reduces=${NUM_REDUCERS_1} \
    -D mapreduce.partition.keycomparator.options="-k2,2nr" \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -combiner "python reducer1.py" \
    -reducer "python reducer1.py" \
    -input ${IN_DIR_1} \
    -output ${OUT_DIR_1} > /dev/null


# Job 2
IN_DIR_2=${OUT_DIR_1}
OUT_DIR_2="second_job_result"
NUM_REDUCERS_2=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR_2} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Second job" \
    -D mapreduce.job.reduces=${NUM_REDUCERS_2} \
    -D mapreduce.partition.keycomparator.options="-k2,2nr" \
    -files mapper2.py,reducer2.py \
    -mapper "python mapper2.py" \
    -reducer "python reducer2.py" \
    -input ${IN_DIR_2} \
    -output ${OUT_DIR_2} > /dev/null

# Get the 7-th most popular word

# hdfs dfs -ls ${OUT_DIR_1}
# hdfs dfs -ls ${OUT_DIR_2}

# hdfs dfs -cat ${OUT_DIR_2}/part-00000 | head

# hdfs dfs -cat ${OUT_DIR_2}/part-00000 | tail
# hdfs dfs -cat ${OUT_DIR_2}/part-00001 | head

hdfs dfs -cat ${OUT_DIR_2}/part-00000 | head -7 | tail -1

is	126420


18/01/19 11:52:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/19 11:52:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/19 11:52:40 INFO mapred.FileInputFormat: Total input files to process : 1
18/01/19 11:52:40 INFO mapreduce.JobSubmitter: number of splits:2
18/01/19 11:52:40 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1516314926040_0054
18/01/19 11:52:41 INFO impl.YarnClientImpl: Submitted application application_1516314926040_0054
18/01/19 11:52:41 INFO mapreduce.Job: The url to track the job: http://37bd34cac093:8088/proxy/application_1516314926040_0054/
18/01/19 11:52:41 INFO mapreduce.Job: Running job: job_1516314926040_0054
18/01/19 11:52:47 INFO mapreduce.Job: Job job_1516314926040_0054 running in uber mode : false
18/01/19 11:52:47 INFO mapreduce.Job:  map 0% reduce 0%
18/01/19 11:53:03 INFO mapreduce.Job:  map 46% reduce 0%
18/01/19 11:53:09 INFO mapreduce.Job:  map 56% reduce 0%
18/01/19 11:53:15 INFO 

## Test

In [7]:
# ! hdfs dfs -cat /data/wiki/en_articles_part/articles-part | head -1

In [8]:
%%writefile debug_data.txt
1	Truong Minh Xuan Thao, Tran Thi Phuong, Ninh Duong Lan Ngoc, Mi Du, Vu Ngoc Tram, Truong Ho Phuong Nga
2	Tran Tam Minh, Minh Beo, Tran Dang Quang Vinh, Quach Vinh Thanh, Ngo Huynh Nhan
3	Nguyen Tan Dung, Truong Tan Sang, Nguyen Phu Trong, Nguyen Minh Triet, Nguyen Tat Thanh, Trinh Xuan Thanh, Dinh La Thang
4	Que Ngoc Hai, Nguyen Cong Phuong

Overwriting debug_data.txt


In [9]:
cat debug_data.txt | python2 ./mapper1.py | sort | python2 ./reducer1.py | head

beo	1
cong	1
dang	1
dinh	1
du	1
dung	1
duong	1
hai	1
ho	1
huynh	1


In [10]:
cat debug_data.txt | python2 ./mapper1.py | sort | python2 ./reducer1.py | python2 ./mapper2.py | sort | python2 ./reducer2.py | head

nguyen	5
minh	4
truong	3
tran	3
thanh	3
phuong	3
ngoc	3
xuan	2
vinh	2
tan	2


In [11]:
cat debug_data.txt | python2 ./mapper1.py | sort | python2 ./reducer1.py | python2 ./mapper2.py | sort | python2 ./reducer2.py | head -7 | tail -1

ngoc	3
